In [124]:
import cv2
import subprocess
from PIL import Image
import numpy as np
from io import BytesIO
import easyocr

reader = easyocr.Reader(['en']) 

# ─── 1. ADB: connect to BlueStacks and grab a screenshot ──────────────────────
device_id = '127.0.0.1:5555'  # change if BlueStacks uses another port
subprocess.run(['adb', 'connect', device_id], check=True)
screenshot = subprocess.check_output(['adb', '-s', device_id, 'exec-out', 'screencap', '-p'])

# Save original screenshot
with open('screenshot.png', 'wb') as f:
    f.write(screenshot)
print('Screenshot saved as screenshot.png')

# ─── 2. Convert bytes to image and crop ────────────────────────────────────────
image = Image.open(BytesIO(screenshot))

# Define the crop box (left, upper, right, lower)
crop_box = (420, 21, 464, 37)  # Adjust as needed
cropped = image.crop(crop_box)

# Save cropped image
cropped.save('cropped.png')
print('Cropped screenshot saved as cropped.png')


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Screenshot saved as screenshot.png
Cropped screenshot saved as cropped.png


In [125]:
import numpy as np

# Load image
img = cv2.imread('cropped.png')  # BGR format
# Create a mask where white pixels are (255, 255, 255)
tolerance = 36

# Define lower and upper bounds for "near white"
lower_bound = np.array([255 - tolerance, 255 - tolerance, 255 - tolerance])
upper_bound = np.array([255, 255, 255])

# Create mask for near-white pixels
white_mask = cv2.inRange(img, lower_bound, upper_bound)

# Set white pixels to black, and others to white
result = np.full_like(img, 255)  # Start with all white
result[white_mask == 255] = [0, 0, 0]  # Set white pixels to black

cv2.imwrite('inverted_white.png', result)
print('Saved as inverted_white.png')

Saved as inverted_white.png


In [126]:
scale_factor = 10
enlarged = cv2.resize(result, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_NEAREST)

cv2.imwrite('enlarged_result.png', enlarged)

True

In [127]:
# Step 2: Run OCR on the enlarged image
results = reader.readtext('cropped.png')  # or use 'cropped_enlarged_bw.png'

# Step 3: Print results
for bbox, text, confidence in results:
    print(f"Detected text: '{text}' with confidence {confidence:.2f}")


Detected text: '6/6' with confidence 0.83


c:\Users\Capta\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
